# 📈 PyKwant Tutorial: Yield Curves and Rates

This notebook introduces the `pykwant.rates` module.

In PyKwant, a Yield Curve is not a complex object with internal state. It is simply a Function (closure) that maps a Date to a Discount Factor:

$$f: \text{Date} \rightarrow \text{DiscountFactor}$$

This functional definition allows us to easily compose curves, shift them for risk analysis (as seen in `risk.py`), or mock them for testing.

## 1. Setup and Imports

In [ ]:
from datetime import date

from pykwant import dates, rates


# Helper to print tables
def print_row(label, val1, val2=None):
    if val2 is None:
        print(f"{label:<20}: {val1:>10.4f}")
    else:
        print(f"{label:<20}: {val1:>10.4f} | {val2:>10.4f}")

## 2. Compounding Logic

Before building curves, we need to understand how rates are compounded. The `compound_factor` function handles this.

* **Continuous**: $e^{rt}$

* **Discrete (Annual)**: $(1+r)^t$

* **Discrete (Semi-Annual)**: $(1 + \frac{r}{2})^{2t}$

In [2]:
rate = 0.05  # 5%
t = 2.0  # 2 Years

print(f"--- Compounding Factors (Rate={rate:.0%}, T={t}y) ---")

# Continuous
cf_cont = rates.compound_factor(rate, t, frequency=0)
print_row("Continuous", cf_cont)

# Annual
cf_ann = rates.compound_factor(rate, t, frequency=1)
print_row("Annual (1/y)", cf_ann)

# Quarterly
cf_q = rates.compound_factor(rate, t, frequency=4)
print_row("Quarterly (4/y)", cf_q)

--- Compounding Factors (Rate=5%, T=2.0y) ---
Continuous          :     1.1052
Annual (1/y)        :     1.1025
Quarterly (4/y)     :     1.1045


## 3. Constructing a Yield Curve

We rarely define curves by formula. Usually, we have a set of market data points (**Pillars**) and we interpolate between them.

We use `create_curve_from_discount_factor`. This factory function performs **Log-Linear Interpolation** on discount factors (equivalent to linear interpolation on zero rates * time).

In [3]:
ref_date = date(2025, 1, 1)

# Market Data (Pillars)
# 1Y: 4.88% implied rate (DF ~ 0.952)
# 2Y: 5.12% implied rate (DF ~ 0.902)
# 5Y: 5.50% implied rate (DF ~ 0.759)
dates_list = [date(2026, 1, 1), date(2027, 1, 1), date(2030, 1, 1)]

dfs_list = [0.952, 0.902, 0.759]

# Create the Curve Function
market_curve = rates.create_curve_from_discount_factor(
    reference_date=ref_date, dates_list=dates_list, dfs_list=dfs_list, day_count=dates.act_365
)

print(f"Curve created. Type: {type(market_curve)}")
# It's a 'function', specifically a closure.

Curve created. Type: <class 'function'>


## 4. Querying the Curve

Now we can treat `market_curve` as a function $f(d)$.

In [4]:
# Check known pillars
print("\n--- Curve Verification (Discount Factors) ---")
print_row("1Y Pillar", market_curve(dates_list[0]))
print_row("5Y Pillar", market_curve(dates_list[2]))

# Interpolation (1.5 Years)
mid_date = date(2026, 7, 2)
print_row("1.5Y (Interp)", market_curve(mid_date))


--- Curve Verification (Discount Factors) ---
1Y Pillar           :     0.9520
5Y Pillar           :     0.7590
1.5Y (Interp)       :     0.9267


## 5. Extracting Zero Rates

Often we want to know the **Zero Rate** (Spot Rate) for a specific maturity, not just the discount factor.

$$r = -\frac{\ln(DF(t))}{t}$$

In [5]:
print("\n--- Zero Rates Profile ---")

# Let's check rates at different maturities
check_dates = [
    date(2026, 1, 1),  # 1Y
    date(2026, 7, 1),  # 1.5Y
    date(2027, 1, 1),  # 2Y
    date(2030, 1, 1),  # 5Y
]

print(f"{'Date':<12} | {'Time (Y)':<8} | {'DF':<8} | {'Zero Rate':<8}")
print("-" * 45)

for d in check_dates:
    # 1. Get DF
    df = market_curve(d)

    # 2. Get Time
    t = dates.act_365(ref_date, d)

    # 3. Get Zero Rate (using helper)
    z_rate = rates.zero_rates(market_curve, ref_date, d)

    print(f"{str(d):<12} | {t:<8.2f} | {df:<8.4f} | {z_rate:.4%}")


--- Zero Rates Profile ---
Date         | Time (Y) | DF       | Zero Rate
---------------------------------------------
2026-01-01   | 1.00     | 0.9520   | 4.9190%
2026-07-01   | 1.50     | 0.9269   | 5.0768%
2027-01-01   | 2.00     | 0.9020   | 5.1570%
2030-01-01   | 5.00     | 0.7590   | 5.5120%


## 6. Forward Rates

The **Forward Rate** is the interest rate implied by the curve for a future period.Example: What is the 1-year rate, starting 1 year from now? ($F_{1y, 2y}$)

$$F = \frac{1}{\tau} \left( \frac{DF_{start}}{DF_{end}} - 1 \right)$$

In [6]:
start_fwd = date(2026, 1, 1)  # 1Y from now
end_fwd = date(2027, 1, 1)  # 2Y from now

fwd_rate = rates.forward_rate(market_curve, start_fwd, end_fwd)

print("\n--- Forward Rates ---")
print(f"Period: {start_fwd} -> {end_fwd}")
print(f"Implied Forward Rate: {fwd_rate:.4%}")

# Interpretation:
# If you lock in a rate for 2026-2027 today, you get ~5.54%.
# This is higher than the spot 1Y rate (4.9%), indicating an upward sloping curve.


--- Forward Rates ---
Period: 2026-01-01 -> 2027-01-01
Implied Forward Rate: 5.5432%


## 7. Discounting Cash Flows

Finally, the primary use of a yield curve is to calculate the **Present Value (PV)** of future cash flows.

In [7]:
cf_amount = 1000.0
payment_date = date(2030, 1, 1)  # 5 Years

pv = rates.present_value(cf_amount, payment_date, market_curve)

print("\n--- Valuation ---")
print(f"Future Value:  {cf_amount:.2f}")
print(f"Payment Date:  {payment_date}")
print(f"Present Value: {pv:.2f}")


--- Valuation ---
Future Value:  1000.00
Payment Date:  2030-01-01
Present Value: 759.00
